# 2 lab

In [ ]:
import pandas as pd

# 0
transactions = pd.read_csv("/content/drive/MyDrive/ITMO/5-sem/ml/transactions.csv", nrows=1000000, index_col='customer_id')
tr_types = pd.read_csv("/content/drive/MyDrive/ITMO/5-sem/ml/tr_types.csv", sep=";", index_col='tr_type')
tr_mcc_codes = pd.read_csv("/content/drive/MyDrive/ITMO/5-sem/ml/tr_mcc_codes.csv", sep=";", index_col='mcc_code')
gender_train = pd.read_csv("/content/drive/MyDrive/ITMO/5-sem/ml/gender_train.csv", sep=",", index_col='customer_id')

print(transactions.head())
print(tr_types.head())
print(tr_mcc_codes.head())
print(gender_train.head())

            tr_datetime  mcc_code  tr_type    amount term_id
customer_id                                                 
39026145     0 10:23:26      4814     1030  -2245.92     NaN
39026145     1 10:19:29      6011     7010  56147.89     NaN
39026145     1 10:20:56      4829     2330 -56147.89     NaN
39026145     1 10:39:54      5499     1010  -1392.47     NaN
39026145     2 15:33:42      5499     1010   -920.83     NaN
                                            tr_description
tr_type                                                   
3200     Плата за предоставление услуг посредством моби...
3210     Плата за предоставление отчета по счету карты ...
3800     Плата за обслуживание банковской карты (за пер...
4000               Плата за получение наличных в Сбербанке
4001     Плата за получение наличных в Сбербанке (в дру...
                                            mcc_description
mcc_code                                                   
742                                     

In [ ]:
# 1.1
"""Для столбца tr_type датафрейма transactions выберите произвольные 1000 строк с помощью метода sample"""
sample = transactions["tr_type"].sample(1000)

# 1.2
"""В полученной на предыдущем этапе подвыборке найдите долю транзакций (стобец tr_description в датасете tr_types), в которой содержится подстрока 'POS' или 'ATM"""
descriptions = tr_types['tr_description']
filtered_types = tr_types[descriptions.str.contains('POS|АТМ')]
merge = sample[sample.isin(filtered_types.index)]


transaction_ratio = len(merge) / len(sample)
print(transaction_ratio)

0.607


In [ ]:
# 2.1
"""Для столбца tr_type датафрейма transactions посчитайте частоту встречаемости всех типов транзакций tr_type в transactions."""
transaction_counts = transactions['tr_type'].value_counts(normalize=True)
print(transaction_counts)

# 2.2
"""Выведите топ-10 транзакций по частоте встречаемости (вывести для них tr_description тоже)."""

top_10_transactions = transaction_counts.head(10)
top_10_descriptions = tr_types.loc[top_10_transactions.index]['tr_description']

result = pd.DataFrame({'tr_type': top_10_transactions.index, 'tr_description': top_10_descriptions.values})
print(result)

1010    0.231117
2010    0.151166
7070    0.149006
1110    0.137658
1030    0.118975
          ...   
2446    0.000004
4096    0.000004
8146    0.000003
1510    0.000002
8100    0.000002
Name: tr_type, Length: 72, dtype: float64
   tr_type                                     tr_description
0     1010                              Покупка. POS ТУ СБ РФ
1     2010              Выдача наличных в АТМ Сбербанк России
2     7070  Перевод на карту (с карты) через Мобильный бан...
3     1110                             Покупка. POS ТУ Россия
4     1030                     Оплата услуги. Банкоматы СБ РФ
5     2370  Списание с карты на карту по операции <перевод...
6     7010       Взнос наличных через АТМ (в своем тер.банке)
7     7030  Перевод на карту (с карты) через АТМ (в предел...
8     7071  Перевод на карту (с карты) через Мобильный бан...
9     1100                                Покупка. ТУ  Россия


In [ ]:
# 3.1
"""В датафрейме transactions найдите клиента с максимальной суммой приходов на карту"""
customer_with_max_income = transactions[transactions['amount'] > 0].groupby('customer_id')['amount'].sum().idxmax()

# 3.2
"""В датафрейме transactions найдите клиента с максимальной суммой расходов по карте"""
customer_with_max_expenses = transactions[transactions['amount'] < 0].groupby('customer_id')['amount'].sum().idxmin()

# 3.3
"""Найдите модуль разницы для этих клиентов между суммой расходов и суммой приходов"""
transactions.loc[customer_with_max_income, 'amount']
# 3.3
"""Найдите модуль разницы для этих клиентов между суммой расходов и суммой приходов"""
income_total = transactions.loc[(transactions.index == customer_with_max_income) & (transactions['amount'] > 0), 'amount'].sum()
expenses_total = transactions.loc[(transactions.index == customer_with_max_expenses) & (transactions['amount'] < 0), 'amount'].sum()
difference = abs(income_total + expenses_total)

print("Customer with max income:", customer_with_max_income)
print("Customer with max expenses:", customer_with_max_expenses)
print("Difference:", difference)

Customer with max income: 70780820
Customer with max expenses: 70780820
Difference: 1837317.980000019


In [ ]:
# 4.1
top_10_transactions = transactions[transactions['tr_type'].isin(transaction_counts.head(10).index)]
mean_top_10 = top_10_transactions.groupby('tr_type')['amount'].mean()
median_top_10 = top_10_transactions.groupby('tr_type')['amount'].median()
pd.concat([mean_top_10, median_top_10], axis=1)

,amount,amount
tr_type,,
1010,-19784.748640,-7411.52
1030,-5320.980222,-2245.92
1100,-44061.827262,-10188.26
1110,-32119.330371,-11207.57
2010,-136077.629325,-44918.32
2370,-205418.249032,-44918.32
7010,276391.789596,112295.79
7030,86104.332909,13951.52
7070,65569.831700,11319.42


In [ ]:
# 4.2.max incomes
trans_clients3 = transactions[transactions.index == customer_with_max_income]  # index is customer_id
trans_clients3_mean = trans_clients3.groupby('tr_type')['amount'].mean()
trans_clients3_median = trans_clients3.groupby('tr_type')['amount'].median()
print(pd.concat([trans_clients3_mean, trans_clients3_median], axis=1))
# 4.2.max expenses
trans_clients3 = transactions[transactions.index == customer_with_max_expenses]  # index is customer_id
trans_clients3_mean = trans_clients3.groupby('tr_type')['amount'].mean()
trans_clients3_median = trans_clients3.groupby('tr_type')['amount'].median()
print(pd.concat([trans_clients3_mean, trans_clients3_median], axis=1))

               amount       amount
tr_type                           
2010    -2.941056e+06 -3368873.660
2330    -2.382398e+06 -2245915.770
2370    -2.218505e+06 -2245915.770
6110     1.756293e+04    10028.010
7020     1.465129e+04    10319.985
7021     2.850067e+04    28500.670
7030     1.406196e+04     8848.910
7031     1.921036e+04    12992.620
7034     1.430648e+04    14306.480
7040     2.153833e+04    14396.320
7070     1.384821e+04     8714.150
7071     1.860057e+04    12543.440
7074     2.281159e+04     9387.930
               amount       amount
tr_type                           
2010    -2.941056e+06 -3368873.660
2330    -2.382398e+06 -2245915.770
2370    -2.218505e+06 -2245915.770
6110     1.756293e+04    10028.010
7020     1.465129e+04    10319.985
7021     2.850067e+04    28500.670
7030     1.406196e+04     8848.910
7031     1.921036e+04    12992.620
7034     1.430648e+04    14306.480
7040     2.153833e+04    14396.320
7070     1.384821e+04     8714.150
7071     1.860057e+0

In [ ]:
transactions = transactions.join(tr_mcc_codes, on='mcc_code', how='inner')
transactions = transactions.join(tr_types, on='tr_type', how='inner')
transactions = transactions.join(gender_train, on='customer_id', how='left')

# Проверяем размерность результирующей таблицы
print(transactions.shape)

(999584, 8)


## Задание 5

In [ ]:
"""Определите модуль разницы между средними тратами женщин и мужчин (трата - отрицательное значение amount)"""
avg_expenses_female = transactions[(transactions['amount'] < 0) & (transactions['gender'] == 0)].groupby('gender')['amount'].mean().abs()
avg_expenses_male = transactions[(transactions['amount'] < 0) & (transactions['gender'] == 1)].groupby('gender')['amount'].mean().abs()
difference_expenses = abs(avg_expenses_female.values[0] - avg_expenses_male.values[0])

print("Разница между средними тратами:", abs(avg_expenses_female.mean() - avg_expenses_male.mean()))

Разница между средними тратами: 32718.054920224655


In [ ]:
"""Определите модуль разницы между средними поступлениями у мужчин и женщин"""
avg_income_female = transactions[(transactions['amount'] > 0) & (transactions['gender'] == 0)].groupby('gender')['amount'].mean()
avg_income_male = transactions[(transactions['amount'] > 0) & (transactions['gender'] == 1)].groupby('gender')['amount'].mean()
difference_income = abs(avg_income_female.values[0] - avg_income_male.values[0])

print("Разница в доходе:", difference_income)

Разница в доходе: 63366.57104801464


## Задание 6

In [ ]:
"""По всем типам транзакций рассчитайте максимальную сумму прихода на карту
(из строго положительных сумм по столбцу amount) отдельно для мужчин и женщин
(назовите ее "max_income").
Оставьте по 10 типов транзакций для мужчин и для женщин,
наименьших среди всех типов транзакций по полученным значениям "max_income"."""
positive_transactions = transactions[transactions['amount'] > 0]
max_income = positive_transactions.groupby(['tr_type', 'gender'])['amount'].max()
max_income_sorted = max_income.sort_values(ascending=True)

top_10_female = max_income_sorted.loc[:, 0].head(10)
top_10_male = max_income_sorted.loc[:, 1].head(10)

print("Максимальные суммы прихода на карту для женщин:")
print(top_10_female)
print("Максимальные суммы прихода на карту для мужчин:")
print(top_10_male)


Максимальные суммы прихода на карту для женщин:
tr_type
4051     1122.96
4210     2245.92
4110     2245.92
2370     2335.75
4100     5041.86
7075     6737.75
2110    22459.16
1010    24839.83
2010    44918.32
2210    68315.82
Name: amount, dtype: float64
Максимальные суммы прихода на карту для мужчин:
tr_type
4110      2245.92
2020      2245.92
1010     28803.87
8100     64682.37
6000     79280.83
7041     87590.72
2010     89836.63
2110    112295.79
2370    113194.15
7015    121279.45
Name: amount, dtype: float64


In [ ]:
"""типы транзакций, которые встречаются одновременно и у мужчин, и у женщин"""
common_types = set(top_10_female.index) & set(top_10_male.index)
common_types

{1010, 2010, 2110, 2370, 4110}

## Задание 8

In [ ]:
"""Из поля tr_datetime выделите час tr_hour, в который произошла транзакция, как первые 2 цифры до ":"."""
transactions['tr_hour'] = transactions['tr_datetime'].str[-8:-6].astype(int)

"""Посчитайте количество транзакций с amount<0 в ночное время для мужчин и женщин. Ночное время - это примерно 00-06 часов."""
night_transactions_female = transactions[(transactions['amount'] < 0) & (transactions['gender'] == 0) & (transactions['tr_hour'] <= 6)]
night_transactions_male = transactions[(transactions['amount'] < 0) & (transactions['gender'] == 1) & (transactions['tr_hour'] >= 0) & (transactions['tr_hour'] <= 6)]

count_night_transactions_female = night_transactions_female.count()[0]
count_night_transactions_male = len(night_transactions_male)

print("Количество транзакций с amount<0 в ночное время для женщин:", count_night_transactions_female)
print("Количество транзакций с amount<0 в ночное время для мужчин:", count_night_transactions_male)

Количество транзакций с amount<0 в ночное время для женщин: 43510
Количество транзакций с amount<0 в ночное время для мужчин: 46694


## Доп. задание

In [ ]:
"""Найдите, какой процент клиентов датасета совершает траты в ночное время"""
transactions['is_night'] = ((transactions['tr_hour'] <= 6) & (transactions['amount'] < 0))
night_clients = transactions[transactions['is_night']].index.nunique()
total_clients = transactions.index.nunique()
(night_clients / total_clients) * 100

90.43340380549682

In [ ]:
"""Посчитайте для этих людей разницу в тратах ночью и не ночью."""
night_transactions = transactions[(transactions['is_night'] == True) & (transactions['amount'] < 0)]
day_transactions = transactions[(transactions['is_night'] == False) & (transactions['amount'] < 0)]

sum_night_expenses = night_transactions.groupby(night_transactions.index)['amount'].sum().abs()
sum_non_night_expenses = day_transactions.groupby(day_transactions.index)['amount'].sum().abs()

abs(sum_night_expenses - sum_non_night_expenses)

customer_id
115617      11629327.18
117556        575978.54
127946              NaN
186730      19916256.47
189257      25912270.64
               ...     
99864691      421255.41
99900908            NaN
99953525     3943747.81
99967537            NaN
99999680    14054419.84
Name: amount, Length: 1892, dtype: float64